## URL extraction

In [27]:
import pandas as pd
import requests
import numpy as np 
from bs4 import BeautifulSoup
import re

data = pd.read_csv('sample_port.csv')

names = data['Scheme Name']



def check_url_for_404(url):
    try:
        response = requests.get(url)
        if response.status_code == 404:
            return True
        else:
            return False
    except requests.exceptions.RequestException as e:
        print("An error occurred:", e)
        return False

# Example usage

scheme_url = []
NAV_list = []

for name in names:
    name1 = name.lower().replace(' ','-').replace('---','-').replace('regular','direct')
    
    
    name2 = name1.replace('plan','growth')

    # remove the string after growth 
    name3 = name2.split('growth', 1)[0].replace('-direct-','-direct-growth').replace('--','-').replace('-idcw-reinvestment','')

    if 'fund-of-fund' in name3:
        name3 = name3.replace('fund-of-fund','fof')

    if name3.endswith('fund-'):
        name3 = name3.replace('fund-','fund-direct-growth')
    
    if name3.endswith('fund'):
        name3 = name3.replace('fund','fund-direct-growth')
    
    if name3.endswith('discovery-'):
        name3 = name3.replace('discovery-','discovery-fund-direct-growth')
    
    if name3=='icici-prudential-large-&-mid-cap-fund-direct-growth':
        name3 = 'icici-prudential-top-100-fund-direct-growth'
    
    if name3=='icici-prudential-bluechip-fund-direct-growth':
        name3 = 'icici-prudential-focused-bluechip-equity-fund-direct-growth' 
    
    if name3=='sbi-large-&-midcap-fund-direct-growth':
        name3 = 'sbi-magnum-multiplier-fund-direct-growth'

    if name3=='kotak-emerging-equity-fund':
        name3 = 'kotak-emerging-equity-scheme-direct-growth'

    if name3=='quant-small-cap-fund-direct-growth':
        name3 = 'quant-small-cap-fund-direct-plan-growth'

    if name3=='dsp-flexi-cap-fund-direct-growth':
        name3 = 'dsp-blackrock-equity-fund-direct-growth'

    if name3=='kotak-emerging-equity-fund-direct-growth':
        name3 = 'kotak-emerging-equity-scheme-direct-growth'

    if name3=='kotak-small-cap-fund-direct-growth':
        name3= 'kotak-midcap-fund-direct-growth'

    if name3=='templeton-india-equity-income-fund-direct-growth':
        name3 = 'templeton-india-equity-i-come-fund-direct-growth'

    if name3=='dsp-banking-&-financial-services-fund-direct-growth':
        name3 = 'dsp-banking-financial-services-fund-direct-growth'

    url = f"https://groww.in/mutual-funds/{name3}"
    print(url)
    is_404 = check_url_for_404(url)
    if is_404:
        print(name3)

        print(f"The URL {url} returns a 404 status code.")
    
    urldat = requests.get(url)

    soup = BeautifulSoup(urldat.text, 'html.parser')



    td_tag = soup.find_all('td',class_="fd12Cell contentPrimary bodyLargeHeavy")[0]

    # Extract the text content from the <td> tag
    value = td_tag.get_text(strip=True)

    NAV= re.search(r'₹([\d.]+)', value).group(1)

    NAV_list.append(NAV)
    scheme_url.append(url)

    #.split('-')[0].lower().replace(' ','-')[:-1])

data['scheme_url'] = np.array(scheme_url)
data['NAV'] = np.array(NAV_list)

data.to_csv('sample_port.csv',index=False)


https://groww.in/mutual-funds/quant-momentum-fund-direct-growth
https://groww.in/mutual-funds/hdfc-transportation-and-logistics-fund-direct-growth
https://groww.in/mutual-funds/quant-momentum-fund-direct-growth
https://groww.in/mutual-funds/axis-india-manufacturing-fund-direct-growth
https://groww.in/mutual-funds/icici-prudential-top-100-fund-direct-growth
https://groww.in/mutual-funds/quant-commodities-fund-direct-growth
https://groww.in/mutual-funds/kotak-healthcare-fund-direct-growth
https://groww.in/mutual-funds/hdfc-mid-cap-opportunities-fund-direct-growth
https://groww.in/mutual-funds/hdfc-technology-fund-direct-growth
https://groww.in/mutual-funds/icici-prudential-focused-bluechip-equity-fund-direct-growth
https://groww.in/mutual-funds/franklin-india-flexi-cap-fund-direct-growth
https://groww.in/mutual-funds/icici-prudential-multicap-fund-direct-growth
https://groww.in/mutual-funds/nippon-india-multi-cap-fund-direct-growth
https://groww.in/mutual-funds/groww-nifty-total-market-i

## Data 

In [2]:
import pandas as pd 

data = pd.read_csv('sample_port.csv')

columns_to_keep = ['Scheme Name', 'ISIN', 'Scheme Category Name', 'Units', 'NAV','scheme_url']

data = data[columns_to_keep]

data.to_csv('sample_port.csv',index=False)
data.head()

,Scheme Name,ISIN,Scheme Category Name,Units,NAV,scheme_url
0,quant Momentum Fund - Regular Plan IDCW - Rein...,INF966L01CY6,Equity - Diversified,3000.000,13.73,https://groww.in/mutual-funds/quant-momentum-f...
1,HDFC Transportation and Logistics Fund Regular...,INF179KC1GX4,Equity - Sector - Auto,1500.000,13.49,https://groww.in/mutual-funds/hdfc-transportat...
2,quant Momentum Fund - Regular Plan Growth - Gr...,INF966L01CW0,Equity - Diversified,7075.965,13.73,https://groww.in/mutual-funds/quant-momentum-f...
3,Axis India Manufacturing Fund Regular Growth,INF846K013S0,Equity - Diversified,3000.000,11.75,https://groww.in/mutual-funds/axis-india-manuf...
4,ICICI Prudential Large & Mid Cap Fund - Growth,INF109K01431,Equity - Diversified,220.345,924.99,https://groww.in/mutual-funds/icici-prudential...


## Data get the holding

In [8]:
import requests
from bs4 import BeautifulSoup
import json


data = pd.read_csv('sample_port.csv')
all_schemes = data['Scheme Name'].unique()

consol_df = pd.DataFrame()
for scheme in all_schemes:
        url = requests.get(data.loc[data['Scheme Name'] == scheme,'scheme_url'].values[0])
        soup = BeautifulSoup(url.text, 'html.parser')

        # Find the script tag with the specific ID
        script_tag = soup.find('script', id='__NEXT_DATA__')

        # Extract the JSON data from the script tag content
        json_data = json.loads(script_tag.contents[0])

        # get holdings
        holdings = json_data['props']['pageProps']['mf']['holdings']

        # get pandas
        hold_df = pd.DataFrame(holdings)
        
        # hold_df = hold_df[['company_name','sector_name','corpus_per']]

        consol_df = pd.concat([hold_df, consol_df], ignore_index=True)

consol_df = consol_df[['company_name','sector_name','corpus_per']]


In [7]:
print(holdings)

[{'scheme_code': '152202', 'portfolio_date': '2024-02-29T00:00:00.570Z', 'company_name': 'Sun Pharmaceutical Industries Ltd.', 'nature_name': 'EQ', 'sector_name': 'Healthcare', 'instrument_name': 'Equity', 'rating': None, 'market_value': 247.773969, 'corpus_per': 6.07418409, 'market_cap': None, 'rating_market_cap': None, 'stock_search_id': 'sun-pharmaceutical-industries-ltd'}, {'scheme_code': '152202', 'portfolio_date': '2024-02-29T00:00:00.570Z', 'company_name': 'Tata Motors Ltd.', 'nature_name': 'EQ', 'sector_name': 'Automobile', 'instrument_name': 'Equity', 'rating': None, 'market_value': 240.4006, 'corpus_per': 5.89342579, 'market_cap': None, 'rating_market_cap': None, 'stock_search_id': 'tata-motors-ltd'}, {'scheme_code': '152202', 'portfolio_date': '2024-02-29T00:00:00.570Z', 'company_name': 'Mahindra & Mahindra Ltd.', 'nature_name': 'EQ', 'sector_name': 'Automobile', 'instrument_name': 'Equity', 'rating': None, 'market_value': 235.7528, 'corpus_per': 5.77948488, 'market_cap': No

In [74]:
import plotly.express as px
import altair as alt 

# Calculate counts of each category
category_counts = data['Scheme Category Name'].value_counts()

# Create a pie chart
fig = px.pie(values=category_counts.values, names=category_counts.index, title='Pie Chart of Categories')


fig.show()

In [76]:

# Calculate counts of each category
# category_counts = df['category'].value_counts().reset_index()

# Rename columns for clarity
category_counts = data['Scheme Category Name'].value_counts().reset_index()
# Rename columns for clarity
category_counts.columns = ['category', 'count']
alt.Chart(category_counts).mark_arc(innerRadius=50).encode(
    theta="count",
    color="category:N",
)

alt.Chart(...)

In [2]:
import pandas as pd
import altair as alt

# Sample dataframes (replace these with your actual data)
df1 = pd.DataFrame({'Company': ['A', 'B', 'C'], 'Portfolio1': [0.3, 0.5, 0.2]})
df2 = pd.DataFrame({'Company': ['A', 'B', 'D'], 'Portfolio2': [0.4, 0.4, 0.2]})

# Merge dataframes on 'Company' column
merged_df = pd.merge(df1, df2, on='Company', how='inner')

# Create scatter plot with overlapping companies
scatter = alt.Chart(merged_df).mark_circle(size=100).encode(
    x=alt.X('Portfolio1', title='Contribution in Portfolio 1'),
    y=alt.Y('Portfolio2', title='Contribution in Portfolio 2'),
    tooltip='Company'
).properties(
    title='Overlapping Companies Between Portfolios'
)

# Create text annotations for company names
text = scatter.mark_text(
    align='left',
    baseline='middle',
    dx=7,  # Adjust text position for better visibility
).encode(
    text='Company'
)

# Combine scatter plot and text annotations
chart = scatter + text

# Show the chart
chart.show()


alt.LayerChart(...)